# Classification after Clustering with Agglomerative Clustering

### Import CSV and Required Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay


warnings.filterwarnings("ignore")
pd.pandas.set_option("display.max_columns", None)
# Create Dataframe
df = pd.read_csv(r"./data_1/clustered_data.csv")
# Print shape of dataset
print(df.shape)

(2240, 22)


**Split X and y**
- Why do we split our data?
> Training Dataset is the part of Original Dataset that we use to train our ML model. The model learns on this data by running the algorithm and maps a function F(x) where “x” in the independent variable (inputs) for “y” where “y” is the dependent variable(output).

In [2]:
X = df.drop("cluster", axis=1) #dropping the target column which is 'cluster'
y = df["cluster"]

## Grid Search

- Why do we use Grid Search?

`GridSearchCV` is a technique to search through the best parameter values from the given set of the grid of parameters. It is basically a cross-validation method. the model and the parameters are required to be fed in. Best parameter values are extracted and then the predictions are made.

## Select the best model
- so here we have some list of the best classification algorithms we imported. Now we will compare each model's score and see which model is performing better than rest of the others

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score,roc_curve,confusion_matrix
from sklearn import metrics 

models = {
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Logistic Regression": LogisticRegression(),
     "K-Neighbors Classifier": KNeighborsClassifier(),
        "AdaBoost Classifier": AdaBoostClassifier()
}

- ### We will create a generic function to check each model's performance so that we can compare those

In [9]:
# Create a function which can evaluate models and return a report 
def evaluate_models(X, y, models):
    '''
    This function takes in X and y and models dictionary as input
    It splits the data into Train Test split
    Iterates through the given model dictionary and evaluates the metrics
    Returns: Dataframe which contains report of all models metrics with cost
    '''
    # separate dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
    

    models_list = []
    scores = []
    
    for i in range(len(list(models))):
        model = list(models.values())[i]
        model.fit(X_train, y_train) # Train model

        # Make predictions
        y_pred = model.predict(X_test)

        score = accuracy_score(y_test,y_pred)
        
        model_name = list(models.keys())[i]
        print(f'---- score for --- {model_name} ----')
        print(f"{score}")
        models_list.append(model_name)
        scores.append(score)
    
    print()
    
    report = pd.DataFrame()
    report['Model_name'] = models_list
    report['Score'] = scores        
    return report

### Let's check the report

In [11]:
report = evaluate_models(X, y, models)

---- score for --- Random Forest ----
0.953125
---- score for --- Decision Tree ----
0.9352678571428571
---- score for --- Gradient Boosting ----
0.9620535714285714
---- score for --- Logistic Regression ----
0.8772321428571429
---- score for --- K-Neighbors Classifier ----
0.8125
---- score for --- AdaBoost Classifier ----
0.9308035714285714



In [13]:
report.sort_values('Score')

,Model_name,Score
4,K-Neighbors Classifier,0.812500
3,Logistic Regression,0.877232
5,AdaBoost Classifier,0.930804
1,Decision Tree,0.935268
0,Random Forest,0.953125
2,Gradient Boosting,0.962054


- ### From the report above we can see that the logistic regression model performed the best, so we will continue training our model using logistic regression algorithm.

### Split into Train and test data

- **Do you know why we split the train and test dataset?**
> The train test split technique can be used for classification and regression problems to test machine learning algorithms. The procedure takes the given dataset and splits it into two subsets: ```Training data/train set:``` it is used to train the algorithm and fit the machine learning model
then we have ```test data/test set``` which is basically a different data for which we know the values but this data was never shown to the model before. Thus if the model after training is performing good on test set as well then we can say that the Machine Learning model is good.

In [15]:
# Separate data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

X_train

,Age,Education,Marital Status,Parental Status,Children,Income,Total_Spending,Days_as_Customer,Recency,Wines,Fruits,Meat,Fish,Sweets,Gold,Web,Catalog,Store,Discount Purchases,Total Promo,NumWebVisitsMonth
994,71,2,0,1,2,64587.0,108.0,4224,49,66,0,16,0.0,6,20.0,1,1,4,2,0,3
2162,68,2,1,1,1,47320.0,414.0,4754,10,200,19,111,50.0,15,19.0,5,1,8,6,0,6
906,64,2,0,0,0,86429.0,1449.0,4437,10,464,28,556,29.0,18,37.0,7,4,7,0,1,2
572,47,1,0,1,1,38593.0,177.0,4506,42,51,12,49,17.0,24,24.0,4,1,3,3,0,8
1877,67,2,1,0,0,72905.0,1515.0,4376,52,407,81,445,120.5,81,126.5,3,7,9,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,56,2,1,1,2,44078.0,41.0,4227,17,24,1,10,2.0,0,4.0,2,0,3,2,0,5
1095,46,2,0,1,1,61825.0,424.0,4573,56,162,50,100,55.0,30,27.0,4,2,8,1,0,4
1130,74,3,1,1,1,67381.0,957.0,4747,67,815,8,53,11.0,0,70.0,2,2,9,4,1,7
1294,62,4,0,1,2,48918.0,62.0,4059,21,52,0,9,0.0,0,1.0,1,0,4,2,0,4


### Let's do hyperparameter tuning
- **And what's it actually?**

> A Machine Learning model is defined as a mathematical model with a number of parameters that need to be learned from the data. By training a model with existing data, we are able to fit the model parameters. 
However, there is another kind of parameter, known as Hyperparameters, that cannot be directly learned from the regular training process. They are usually fixed before the actual training process begins. These parameters express important properties of the model such as its complexity or how fast it should learn. 

In [ ]:
# Grid search cross-validation
from sklearn.model_selection import GridSearchCV

params = {
    "n_estimators": np.arange(50, 301, 50),     # number of trees
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "max_depth": [2, 3, 4, 5],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "subsample": [0.6, 0.8, 1.0],
    "max_features": ["sqrt", "log2", None]
}

gdbc=GradientBoostingClassifier()

gdbc_cv=GridSearchCV(gdbc,params,cv=10)
gdbc_cv.fit(X_train,y_train)

print("tuned hpyerparameters :(best parameters) ",gdbc_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

### Let's now train the model 

In [18]:
gdbc=GradientBoostingClassifier()

gdbc.fit(X_train,y_train)
y_pred = gdbc.predict(X_test)

**Initialize model with best parameters**

### Let's check the report now

In [20]:
# best_model = best_lr_model.fit(X_train,y_train)
# y_pred = best_model.predict(X_test)
score = accuracy_score(y_test,y_pred)
cr = classification_report(y_test,y_pred)

print(" Gradient Boosting ")
print("Accuracy Score value: {:.4f}".format(score))
print(cr)

 Gradient Boosting 
Accuracy Score value: 0.9524
              precision    recall  f1-score   support

           0       0.95      0.94      0.95       178
           1       0.98      0.97      0.97       263
           2       0.92      0.94      0.93       231

    accuracy                           0.95       672
   macro avg       0.95      0.95      0.95       672
weighted avg       0.95      0.95      0.95       672



## Confusion matrix of the model
- **What is confusion matrix ?**
> The confusion matrix is a matrix used to determine the performance of the classification models for a given set of test data. It can only be determined if the true values for test data are known. The matrix itself can be easily understood, but the related terminologies may be confusing.

- **Reports**

**We can see, that the model performed pretty well.**
- we have used Gradient Boosting as it performed well than other models
- We got a good accuracy while predicting the test dataset.

X_test

In [24]:
X_test

,Age,Education,Marital Status,Parental Status,Children,Income,Total_Spending,Days_as_Customer,Recency,Wines,Fruits,Meat,Fish,Sweets,Gold,Web,Catalog,Store,Discount Purchases,Total Promo,NumWebVisitsMonth
324,72,2,0,1,1,40464.0,630.0,4457,78,424,17,118,7.0,23,41.0,8,2,8,6,0,8
96,65,2,0,1,1,47916.0,606.0,4801,72,505,0,26,0.0,0,75.0,7,4,6,5,1,6
2104,53,0,1,0,0,14188.0,75.0,4703,40,2,7,11,16.0,12,27.0,2,0,4,1,0,6
1259,56,2,1,0,0,76653.0,2279.0,4534,91,736,63,556,120.5,81,126.0,4,7,11,1,2,2
1061,67,2,1,1,2,65196.0,1155.0,4556,34,743,19,181,12.0,0,126.5,7,6,11,2,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064,51,2,1,1,1,67445.0,1174.0,4785,63,757,80,217,29.0,80,11.0,9,6,12,5,0,6
1735,71,2,0,1,1,70044.0,1504.0,4713,46,1073,0,250,120.5,14,14.0,7,10,5,4,0,5
1057,74,4,1,0,0,53312.0,278.0,4511,32,241,0,12,0.0,0,25.0,5,1,5,1,0,7
1461,66,2,1,1,1,57957.0,637.0,4531,24,290,59,177,77.0,5,29.0,4,6,8,7,0,3
